# GreenValueNet

This notebook contains the code needed to execute the GreenValueNet hedonic pricing neural network. 

### Set up and data loading

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from data_load_funcs import get_params, load_data_catalogue
from processing_funcs import process_data, normalise_values
from model_funcs import *

params = get_params()
data_catalogue = load_data_catalogue()

If you do not have a file called `dataset.csv` in the `data/interim_files` folder the following cell will generate this folder and generate summary statistics. The data processing happens locally and invovles large datasets with spatial components so can take quite several hours - please be pateint! If you already have the file, it will be read in and summary statistics are generated. 

In [ ]:
dataset = process_data(data_catalogue, params)

# show summary stats
summary_stats = dataset.describe().transpose()[['mean', 'std', 'max', 'min']]
summary_stats.columns = ['Mean', 'Std Dev', 'Maximum', 'Minimum']
print(summary_stats)

Now we normalise any non-encoded variables to increase speed of learning of algorithm and convert the dataset to an array of inputs, and an associated output array.

In [ ]:
norm_cols = [col for col in dataset.columns if col not in params['non_norm_cols']]
for col in norm_cols:
    dataset[col] = normalise_values(dataset[col])

# creates an arry of shape m, x, y
x, y = create_x_y_arr(dataset, params)

The dataset is then split into train, dev and test sets using sci-kit learn.

In [ ]:
x_train, x_dev, x_test, y_train, y_dev, y_test = split_to_test_dev_train(
    x,
    y,
    params['dev_size'],
    params['test_size'],
    prop=False
)

## Benchmarking

To evaluate the performance of my neural network I will run random forest and XGBoost regressions as baseline models. I will then build 2 alternative models: a deep neural network and a bayesian model. We optimise based on the mean squared error (MSE) but and report this as our measure of performance.

### Random Forest


In [ ]:
# run baseline random forest regression using scikit-learn
rfr_model = random_forest_reg(
    x_train,
    y_train,
    tuning=False
)

# now run with grid search to tune parameters
rfr_tuned  =  random_forest_reg(
    x_train,
    y_train,
    tuning=True,
    tuning_params = params['tuning_dict']['grid']
)

# generate predictions and measure according to mean squared error
rfr_pred, rfr_mse = generate_pred_metric(rfr_model, mean_squared_error, x_dev, y_dev)
rfr_t_pred, rfr_t_mse = generate_pred_metric(rfr_tuned, mean_squared_error, x_dev, y_dev)

### XGBoost

In [ ]:
xgb_model = boosted_grad_reg(x_train, y_train)
xgb_pred, xgb_mse = generate_pred_metric(xgb_model, mean_squared_error, x_dev, y_dev)

## Neural networks

We know build some neural networks. Number of epochs, hidden layers, and nodes in hidden layers is initially set with rules of thumb but then optimiszed using hyperparameter tuning.

In [ ]:
# set epochs to be 3 times number of features
epochs = int(x_train.shape[1]) * 3

# set n_hidden_units to be mean of input and output layer sizes
n_hidden_units = round((x_train.shape[1] + 1) / 2)

### Single Layer Neural Network

A single hidden layer with ReLU activation is used with a linear output layer.

In [88]:
single_nn = neural_net(
    x_train,
    y_train,
    n_hidden_units = n_hidden_units,
    epochs = epochs,
    validation_data = (x_dev, y_dev)
)

Epoch 1/69
4/4 [==============================] - 1s 96ms/step - loss: 136.3968 - mean_squared_error: 136.3968 - val_loss: 85.3140 - val_mean_squared_error: 85.3140
Epoch 2/69
4/4 [==============================] - 0s 16ms/step - loss: 46.1918 - mean_squared_error: 46.1918 - val_loss: 14.7610 - val_mean_squared_error: 14.7610
Epoch 3/69
4/4 [==============================] - 0s 16ms/step - loss: 16.5098 - mean_squared_error: 16.5098 - val_loss: 24.5878 - val_mean_squared_error: 24.5878
Epoch 4/69
4/4 [==============================] - 0s 17ms/step - loss: 24.6024 - mean_squared_error: 24.6024 - val_loss: 9.8073 - val_mean_squared_error: 9.8073
Epoch 5/69
4/4 [==============================] - 0s 15ms/step - loss: 5.9552 - mean_squared_error: 5.9552 - val_loss: 15.4586 - val_mean_squared_error: 15.4586
Epoch 6/69
4/4 [==============================] - 0s 13ms/step - loss: 10.5398 - mean_squared_error: 10.5398 - val_loss: 17.0714 - val_mean_squared_error: 17.0714
Epoch 7/69
4/4 [========

In [91]:
single_nn.history.history

{'loss': [136.3968048095703,
  46.19175720214844,
  16.509822845458984,
  24.60244369506836,
  5.955211639404297,
  10.539754867553711,
  8.010050773620605,
  2.6183974742889404,
  4.186127185821533,
  2.4716479778289795,
  1.7420923709869385,
  2.2516214847564697,
  1.3061832189559937,
  1.6037441492080688,
  1.327576994895935,
  1.1223970651626587,
  1.1854450702667236,
  0.9423322081565857,
  1.0089224576950073,
  0.8746951222419739,
  0.8565689325332642,
  0.8118577003479004,
  0.7847859263420105,
  0.764011800289154,
  0.7308076024055481,
  0.7014819979667664,
  0.6861767172813416,
  0.6691266298294067,
  0.6542291641235352,
  0.6371952891349792,
  0.6249568462371826,
  0.6107545495033264,
  0.5932189226150513,
  0.5858515501022339,
  0.5719519257545471,
  0.5639655590057373,
  0.5509668588638306,
  0.5444613695144653,
  0.534468412399292,
  0.5252289772033691,
  0.5187070369720459,
  0.5070637464523315,
  0.5148545503616333,
  0.4976750910282135,
  0.4963563084602356,
  0.5127622

### Deep Neural Network

The full model is specified as a deep neural network using layers with ReLU activation functions with a linear activation in the output layer. The choice of number of layers was initially kept small due to computational processing constraints.

In [92]:
deep_nn = neural_net(
    x_train,
    y_train,
    n_layers = 5,
    n_hidden_units = n_hidden_units,
    epochs = epochs,
    validation_data = (x_dev, y_dev)
)

Epoch 1/69
4/4 [==============================] - 1s 82ms/step - loss: 100.1980 - mean_squared_error: 100.1980 - val_loss: 32.5866 - val_mean_squared_error: 32.5866
Epoch 2/69
4/4 [==============================] - 0s 17ms/step - loss: 24.8172 - mean_squared_error: 24.8172 - val_loss: 28.3076 - val_mean_squared_error: 28.3076
Epoch 3/69
4/4 [==============================] - 0s 16ms/step - loss: 11.7905 - mean_squared_error: 11.7905 - val_loss: 7.6204 - val_mean_squared_error: 7.6204
Epoch 4/69
4/4 [==============================] - 0s 15ms/step - loss: 5.5939 - mean_squared_error: 5.5939 - val_loss: 9.1073 - val_mean_squared_error: 9.1073
Epoch 5/69
4/4 [==============================] - 0s 17ms/step - loss: 4.1919 - mean_squared_error: 4.1919 - val_loss: 2.2644 - val_mean_squared_error: 2.2644
Epoch 6/69
4/4 [==============================] - 0s 14ms/step - loss: 2.8088 - mean_squared_error: 2.8088 - val_loss: 2.7438 - val_mean_squared_error: 2.7438
Epoch 7/69
4/4 [==================

In [ ]:
# save model as tensor object and stick in a folder called outputs
model_dir = cwd / "outputs" / "models"
deep_nn.export(model_dir / "deep_nn.tf") # check this file ending